In [302]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [303]:

internal_urls = set()
external_urls = set()

In [304]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [305]:

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls


In [306]:
total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [307]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://www.vanderbilt.edu/coronavirus/"
internal_urls.add(base_url)

In [308]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='www.vanderbilt.edu', path='/coronavirus/', params='', query='', fragment='')
https://www.vanderbilt.edu
www.vanderbilt.edu/coronavirus/
www.vanderbilt.edu


In [309]:
# internal_urls

In [310]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-undergraduate-students/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-graduate-and-professional-students/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-international-students/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-faculty/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-postdocs/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-staff/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-visitors-and-suppliers/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/tag/messages-from-interim-chancellor/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/tag/messages-from-faculty-affairs/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/tag/messages-from-dean-of-students/
[INT] Internal li

[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/updated-services/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/resources/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/telemedicine/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/helping-others/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/satellite-services/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/your-health-insurance/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/your-health-insurance/benefits-and-premium-rates/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/your-health-insurance/enroll/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/waive/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/health-insurance-glossary/
[INT] Internal link: https://www.vanderbilt.edu/studentcarenetwork/leaving-vanderbilt/
[INT] Internal link: https:

[INT] Internal link: https://www.vanderbilt.edu/provost/committees/closed/
[INT] Internal link: https://www.vanderbilt.edu/provost/committees/closed/art-governance/
[INT] Internal link: https://www.vanderbilt.edu/provost/committees/closed/assistant-provost-search-committee/
[INT] Internal link: https://www.vanderbilt.edu/provost/committees/closed/classroom-tech/
[INT] Internal link: https://www.vanderbilt.edu/provost/committees/closed/blair-internal-review-committee/
[INT] Internal link: https://www.vanderbilt.edu/provost/committees/closed/businessminor/
[INT] Internal link: https://www.vanderbilt.edu/provost/committees/closed/business_minor_committee/
[INT] Internal link: https://www.vanderbilt.edu/chancellor/facultyengagement/committees-humanities/
[INT] Internal link: https://www.vanderbilt.edu/provost/committees/closed/commencement/
[INT] Internal link: https://www.vanderbilt.edu/provost/committees/closed/facultyvoices/
[INT] Internal link: https://www.vanderbilt.edu/provost/commit

[INT] Internal link: http://www.vanderbilt.edu/facultysenate/
[INT] Internal link: http://www.vanderbilt.edu/facultysenate/index.php
[INT] Internal link: http://www.vanderbilt.edu/facultysenate/2020-spring-assembly/
[INT] Internal link: http://www.vanderbilt.edu/facultysenate/constitution-documents/index.php
[INT] Internal link: http://www.vanderbilt.edu/facultysenate/constitution-documents/geninfo-constitution.php
[INT] Internal link: http://www.vanderbilt.edu/facultysenate/constitution-documents/rules-of-order.php
[INT] Internal link: http://www.vanderbilt.edu/boardoftrust/bylaws/
[INT] Internal link: http://www.vanderbilt.edu/facultysenate/constitution-documents/new-degree-process.php
[INT] Internal link: http://www.vanderbilt.edu/facultysenate/constitution-documents/senate-newsletter.php
[INT] Internal link: http://www.vanderbilt.edu/facultysenate/files/academic-freedom-of-expression-8-17-16.pdf
[INT] Internal link: http://www.vanderbilt.edu/facultysenate/members/
[INT] Internal li

[INT] Internal link: http://www.vanderbilt.edu/magazines/vanderbilt-business/
[INT] Internal link: http://www.vanderbilt.edu/magazines/vanderbilt-engineering/
[INT] Internal link: http://www.vanderbilt.edu/viee/
[INT] Internal link: http://www.vanderbilt.edu/vicb
[INT] Internal link: http://www.vanderbilt.edu/VICBC/
[INT] Internal link: http://www.vanderbilt.edu/magazines/vanderbilt-magazine/
[INT] Internal link: http://www.vanderbilt.edu/university-press/
[INT] Internal link: http://www.vanderbilt.edu/vanderbiltview/
[INT] Internal link: http://www.vanderbilt.edu/vwc/
[INT] Internal link: http://www.vanderbilt.edu/vector/ 
[INT] Internal link: http://www.vanderbilt.edu/provost/
[INT] Internal link: http://www.vanderbilt.edu/divadm/
[INT] Internal link: http://www.vanderbilt.edu/generalcounsel/
[INT] Internal link: http://www.vanderbilt.edu/publicaffairs/
[INT] Internal link: http://www.vanderbilt.edu/studentcenters/services/productionservices/
[INT] Internal link: http://www.vanderbil

[INT] Internal link: https://www.vanderbilt.edu/vuarts/about/
[INT] Internal link: https://www.vanderbilt.edu/vuarts/about/arts-at-vu-listserv/
[INT] Internal link: https://www.vanderbilt.edu/vuarts/about/contact/
[INT] Internal link: https://www.vanderbilt.edu/vuarts/about/directions-parking/
[INT] Internal link: https://www.vanderbilt.edu/vuarts/about/galleries-exhibition-spaces/
[INT] Internal link: http://www.vanderbilt.edu/artscreative/
[INT] Internal link: https://www.vanderbilt.edu/vuarts/about/tickets/
[INT] Internal link: https://www.vanderbilt.edu/vuarts/students/
[INT] Internal link: https://www.vanderbilt.edu/vuarts/faculty/
[INT] Internal link: http://www.vanderbilt.edu/gallery/
[INT] Internal link: http://www.vanderbilt.edu/internationallens/
[INT] Internal link: http://www.vanderbilt.edu/artscreative
[INT] Internal link: http://www.vanderbilt.edu/sarrattgallery/
[INT] Internal link: http://www.vanderbilt.edu/sarrattart/
[INT] Internal link: http://www.vanderbilt.edu/sarr

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] Total External links: 513
[+] Total Internal links: 786
[+] Total: 1299


In [ ]:
count = 0
last_count = 989874
immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

for i in range(0,4):
    for url in internal_urls:
        if base_url_text in url:
            immediate_urls.add(url)
    count = len(immediate_urls)
    print(count,i)
    if(count == last_count):
        break
    last_count = count
    for immediate_url in immediate_urls:
        try:
            crawl(immediate_url)
            print("[+] NEW Total External links:", len(external_urls))
            print("[+] NEW Total Internal links:", len(internal_urls))
            print("[+] NEW Total:", len(external_urls) + len(internal_urls))
            if len(internal_urls)>1000:
                break
        except requests.exceptions.ConnectionError:
            continue

66 0
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/author/webcomm/
[INT] Internal link: https://www.vanderbilt.edu/brightspace/2020/03/06/putting-some-of-your-course-content-online-in-a-hurry-we-have-resources-for-you/
[+] NEW Total External links: 515
[+] NEW Total Internal links: 788
[+] NEW Total: 1303
[+] NEW Total External links: 515
[+] NEW Total Internal links: 788
[+] NEW Total: 1303
[+] NEW Total External links: 515
[+] NEW Total Internal links: 788
[+] NEW Total: 1303
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-resources/fall-2020-graduate-and-professional-academic-calendars/
[INT] Internal link: https://www.vanderbilt.edu/faculty-affairs/faculty-adaptive-learning-resources/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/travel-information/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/faqs/interior-and-exterior-covid-signage/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/faqs/safety-gu

[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-resources/anchordown-stepup/mask-with-person-1024x1024/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-resources/anchordown-stepup/social-distance-300x300/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-resources/anchordown-stepup/handwashing-300x300/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-resources/anchordown-stepup/temperature-300x300/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/faqs/student-symptom-monitoring/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/fall-2020/fall-2020-resources/resources-for-vu-communicators/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/faqs/testing-symptom-monitoring-contact-tracing/
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/faqs/accommodations-process-for-those-in-high-risk-health-groups/
[INT] Internal link: ht

[+] NEW Total External links: 688
[+] NEW Total Internal links: 891
[+] NEW Total: 1579
[+] NEW Total External links: 688
[+] NEW Total Internal links: 891
[+] NEW Total: 1579
[+] NEW Total External links: 688
[+] NEW Total Internal links: 891
[+] NEW Total: 1579
[+] NEW Total External links: 688
[+] NEW Total Internal links: 891
[+] NEW Total: 1579
[+] NEW Total External links: 692
[+] NEW Total Internal links: 891
[+] NEW Total: 1583
[INT] Internal link: https://www.vanderbilt.edu/coronavirus/author/titel/
[+] NEW Total External links: 692
[+] NEW Total Internal links: 892
[+] NEW Total: 1584
[+] NEW Total External links: 692
[+] NEW Total Internal links: 892
[+] NEW Total: 1584
[+] NEW Total External links: 692
[+] NEW Total Internal links: 892
[+] NEW Total: 1584
148 1
[+] NEW Total External links: 692
[+] NEW Total Internal links: 892
[+] NEW Total: 1584
[+] NEW Total External links: 692
[+] NEW Total Internal links: 892
[+] NEW Total: 1584
[+] NEW Total External links: 692
[+] NE

[+] NEW Total External links: 702
[+] NEW Total Internal links: 912
[+] NEW Total: 1614
[+] NEW Total External links: 702
[+] NEW Total Internal links: 912
[+] NEW Total: 1614
[+] NEW Total External links: 702
[+] NEW Total Internal links: 912
[+] NEW Total: 1614
[+] NEW Total External links: 702
[+] NEW Total Internal links: 912
[+] NEW Total: 1614
[+] NEW Total External links: 702
[+] NEW Total Internal links: 912
[+] NEW Total: 1614
[+] NEW Total External links: 702
[+] NEW Total Internal links: 912
[+] NEW Total: 1614
[+] NEW Total External links: 702
[+] NEW Total Internal links: 912
[+] NEW Total: 1614
[INT] Internal link: https://www.vanderbilt.edu/ohare/move-out-instructions-and-information/
[+] NEW Total External links: 702
[+] NEW Total Internal links: 913
[+] NEW Total: 1615
[+] NEW Total External links: 702
[+] NEW Total Internal links: 913
[+] NEW Total: 1615
[+] NEW Total External links: 702
[+] NEW Total Internal links: 913
[+] NEW Total: 1615
[+] NEW Total External link

[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
169 2
[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
[+] NEW Total External links: 724
[+] NEW Total Internal links: 921
[+] NEW Total: 1645
[INT] Internal link: https://www.vanderbilt.edu/geo/study-abroad-coronavirus-faq/
[+] NEW Total External links: 72

[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 731
[+] NEW Total Internal links: 925
[+] NEW Total: 1656
[+] NEW Total External links: 73

In [ ]:
def removeAfterN(yourStr, nth, occurenceOf):
    return occurenceOf.join(yourStr.split(occurenceOf)[:nth])

In [ ]:
semifinal_immediate_urls = set()

for immediate_url in immediate_urls:
    stripped_url = removeAfterN(immediate_url,2,"http")
    
    
    semifinal_immediate_urls.add(stripped_url)
#     final_immediate_urls.add(rest)
#     print(rest)


    
for i in semifinal_immediate_urls:
    print(i)
total_count = len(semifinal_immediate_urls)
print(total_count)

In [ ]:
final_immediate_urls = set()
for semifinal_immediate_url in semifinal_immediate_urls:
    if "tel:" not in semifinal_immediate_url:
        final_immediate_urls.add(semifinal_immediate_url)


for i in final_immediate_urls:
    print(i)
total_count = len(final_immediate_urls)
print(total_count)       

In [ ]:

parent_folder = "Doctoral" #Change based on folder
name_of_folder = f'{base_url_text_domain}_Screenshots'
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
    os.makedirs(f'{parent_folder}\\{name_of_folder}')

In [ ]:
webpage_save = set()
webpage_save_count = 0
webpage_cannot_save = 0
for i in final_immediate_urls:
    if ' ' in i:
        continue
    try:
        code=urlopen(i).getcode()
        webpage_save.add(i)
        webpage_save_count = webpage_save_count+1
        print(f"{code} can_save - {i}")
    except (urllib.error.HTTPError,urllib.error.URLError) as e:
        print("discarded")
        webpage_cannot_save = webpage_cannot_save+1
        continue

print(f'Webpages save as HTML :{webpage_save_count}')
print(f'Webpages unable to save as HTML :{webpage_cannot_save}')
        

In [ ]:
pdf_count = 0
pdf_urls = set()
for webpage in webpage_save:
    try:
        if webpage[-4:] == ".pdf":
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                file_name = "Home"
            pdf_urls.add(webpage)
            print(webpage)
            r = requests.get(url, stream=True)
            with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                for chunk in r.iter_content(2000):
                    fd.write(chunk)
            pdf_count = pdf_count + 1
    except:
        print("caught")

print(f"Pdfs saved: {pdf_count}")

In [ ]:
final_immediate_urls = final_immediate_urls - pdf_urls
webpage_save = webpage_save - pdf_urls
web_save_count = len(webpage_save)
total_count = len(final_immediate_urls)
print(total_count) 

In [ ]:

#     try:
#         print(i)
        
#         print("added")
#     except urllib.error.HTTPError as e:
#         if e.code in (400,403):
#             print("discarded")
#             continue

In [ ]:
final_webpage_save_count = webpage_save_count - pdf_count
print(final_webpage_save_count)

In [ ]:
webpage_save_counter_loop = 0
for webpage in webpage_save:
    try:
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
        webpage_save_counter_loop = webpage_save_counter_loop+1
        print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
    except:
        print("caught")

print("Saving Completed")

In [ ]:
count = 0
for i in final_immediate_urls:
    try:
        url = i
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        count = count + 1
        print(f"{count} of {total_count}")
        print(f'Visiting {base_url_text}/{file_name}')
        print(f'...Taking a screenshot')
        driver.get(url)
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
        print(f'Screenshot of {file_name} page taken! \n')
    except:
        print("caught")
#     print("Saving HTML")
#     pyautogui.hotkey('ctrl', 's')
#     time.sleep(1)
#     pyautogui.typewrite(f'{file_name}.html')
#     pyautogui.hotkey('enter')
driver.quit()
    
print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')